In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

2023-12-09 21:18:24,455 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-12-09 21:18:24,457 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2023-12-09 21:18:24,457 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2023-12-09T21:18:24.457336', 'gensim': '4.3.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'created'}


Creating a transformation:

In [3]:
from gensim import models

tfidf = models.TfidfModel(corpus)  # step 1 -- initialize a model

2023-12-09 21:23:32,915 : INFO : collecting document frequencies
2023-12-09 21:23:32,915 : INFO : PROGRESS: processing document #0
2023-12-09 21:23:32,916 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 9 documents and 12 features (28 matrix non-zeros)', 'datetime': '2023-12-09T21:23:32.916044', 'gensim': '4.3.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'initialize'}


tfidf is treated as a read-only object that can be used to convert any vector from the old representation (bag-of-words integer counts) to the new representation (TfIdf real-valued weights):

In [4]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow])  # step 2 -- use the model to transform vectors

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [5]:
# to apply a transformation to a whole corpus:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [7]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
#Here we transformed our Tf-Idf corpus via Latent Semantic Indexing into a latent 2-D space (2-D because we set num_topics=2).

2023-12-09 21:32:44,795 : INFO : using serial LSI version on this node
2023-12-09 21:32:44,796 : INFO : updating model with new documents
2023-12-09 21:32:44,796 : INFO : preparing a new chunk of documents
2023-12-09 21:32:44,797 : INFO : using 100 extra samples and 2 power iterations
2023-12-09 21:32:44,797 : INFO : 1st phase: constructing (12, 102) action matrix
2023-12-09 21:32:44,798 : INFO : orthonormalizing (12, 102) action matrix
2023-12-09 21:32:44,799 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2023-12-09 21:32:44,800 : INFO : computing the final decomposition
2023-12-09 21:32:44,800 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2023-12-09 21:32:44,800 : INFO : processed documents up to #9
2023-12-09 21:32:44,801 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2023-12-09 21:32:44,802 : INFO : topic #

In [8]:
lsi_model.print_topics(2)

2023-12-09 21:33:19,196 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2023-12-09 21:33:19,197 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

It appears that according to LSI, “trees”, “graph” and “minors” are all related words (and contribute the most to the direction of the first topic), while the second topic practically concerns itself with all the other words. As expected, the first five documents are more strongly related to the second topic while the remaining four documents to the first topic:

In [9]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc, as_text in zip(corpus_lsi, documents):
    print(doc, as_text)

[(0, 0.06600783396090153), (1, -0.520070330636186)] Human machine interface for lab abc computer applications
[(0, 0.19667592859142388), (1, -0.7609563167700055)] A survey of user opinion of computer system response time
[(0, 0.08992639972446197), (1, -0.7241860626752515)] The EPS user interface management system
[(0, 0.07585847652177953), (1, -0.6320551586003433)] System and human system engineering testing of EPS
[(0, 0.10150299184980091), (1, -0.573730848300296)] Relation of user perceived response time to error measurement
[(0, 0.7032108939378313), (1, 0.16115180214025646)] The generation of random binary unordered trees
[(0, 0.8774787673119837), (1, 0.1675890686465923)] The intersection graph of paths in trees
[(0, 0.9098624686818584), (1, 0.14086553628718823)] Graph minors IV Widths of trees and well quasi ordering
[(0, 0.6165825350569285), (1, -0.05392907566389496)] Graph minors A survey


In [10]:
import os
import tempfile

with tempfile.NamedTemporaryFile(prefix='model-', suffix='.lsi', delete=False) as tmp:
    lsi_model.save(tmp.name)  # same for tfidf, lda, ...

loaded_lsi_model = models.LsiModel.load(tmp.name)

os.unlink(tmp.name)

2023-12-09 21:34:36,620 : INFO : Projection lifecycle event {'fname_or_handle': 'C:\\Users\\Lenovo\\AppData\\Local\\Temp\\model-m4wq1v8r.lsi.projection', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-12-09T21:34:36.620238', 'gensim': '4.3.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'saving'}
2023-12-09 21:34:36,622 : INFO : saved C:\Users\Lenovo\AppData\Local\Temp\model-m4wq1v8r.lsi.projection
2023-12-09 21:34:36,622 : INFO : LsiModel lifecycle event {'fname_or_handle': 'C:\\Users\\Lenovo\\AppData\\Local\\Temp\\model-m4wq1v8r.lsi', 'separately': 'None', 'sep_limit': 10485760, 'ignore': ['projection', 'dispatcher'], 'datetime': '2023-12-09T21:34:36.622229', 'gensim': '4.3.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'saving'}
2023-12-09 21:34:36,622 : INFO : not storing a